### Vulnerability Analysis with Nmap and Vulners

##### Nmap + Vulners Vulnerability Scan

Objective:

Perform a network vulnerability assessment by combining Nmap with the Vulners API. This process demonstrates how to:

- Run a network scan using Nmap.
- Fetch vulnerability data based on the detected services.
- Analyze and generate detailed vulnerability reports.


#### Table of Contents

1. [Install Required Libraries](#install-required-libraries)
2. [Import Libraries](#import-libraries)
3. [Setup Nmap and Vulners API](#setup-nmap-and-vulners-api)
4. [Define Target and Run Nmap Scan](#define-target-and-run-nmap-scan)
5. [Query Vulners API for Vulnerabilities](#query-vulners-api-for-vulnerabilities)
6. [Generate Reports (Full Vulnerability Report)](#generate-reports-full-vulnerability-report)
7. [Service-Specific Vulnerability Reports](#service-specific-vulnerability-reports)
8. [Severity-Based Reports (High & Critical CVSS Scores)](#severity-based-reports-high-critical-cvss-scores)
9. [Export Reports in CSV Format](#export-reports-in-csv-format)




Step 1: Install Required Libraries
- Why?: To run Nmap and query the Vulners API, you need to install two key Python libraries:
  - `python-nmap` to interact with Nmap.
  - `vulners` to interact with the Vulners API.
  
- Instructions: In this step, you’ll install both libraries using `pip`. This command will ensure you have everything set up for scanning and querying vulnerabilities.
python

In [1]:
# Install Nmap and Vulners libraries using pip
!pip install python-nmap vulners

- Explanation: The !pip install command installs the necessary packages to run Nmap scans and access the Vulners API from Python.

---

Step 2: Import Libraries
- Why?: After installing the required libraries, you need to import them into the notebook so you can use their functionality in subsequent steps.

- Instructions: Import the nmap and vulners libraries, along with the json library for formatting output.

In [2]:
import nmap
import vulners
import json

- Explanation:
  - nmap is used for performing network scans.
  - vulners allows querying the Vulners API for vulnerabilities.
  - json helps format and display the vulnerability data in a readable way.

---

##### Step 3: Setup Nmap and Vulners API
- Why?: You need to initialize the Nmap scanner to perform network scans, and you must authenticate with the Vulners API using your API key to access vulnerability data.

- Instructions: Replace the placeholder API key with your Vulners API key. This key authenticates your requests to the Vulners database.

In [3]:
# Initialize Nmap scanner and Vulners API
nm = nmap.PortScanner()
vulners_api = vulners.VulnersApi(api_key='YOUR_VULNERS_API_KEY')  # Replace with your API key

- Explanation:
  - Nmap PortScanner: The Nmap scanner (nm) will handle the network scanning part.
  - Vulners API: The vulners_api object will query the Vulners database for vulnerabilities related to  detected services.

---

Step 4: Define Target and Run Nmap Scan

- Why?: This step defines the target IP address or hostname that you want to scan using Nmap. You’ll run a SYN scan to discover open ports and services.

- Instructions: Replace the target IP (`127.0.0.1`) with the IP or hostname you want to scan. The scan will check for open ports and running services.

In [ ]:
# Define the target (replace with the IP address or hostname you want to scan)
target = '127.0.0.1'  # Replace with your target

# Run an Nmap SYN scan on ports 1-1000
print(f"Starting Nmap scan on {target}")
nm.scan(target, arguments='-sS -p 1-1000')  # Scan the first 1000 ports

# Display the scan results
for host in nm.all_hosts():
    print(f"\nHost: {host} ({nm[host].hostname()})")
    print(f"State: {nm[host].state()}")
    for proto in nm[host].all_protocols():
        ports = nm[host][proto].keys()
        for port in ports:
            service = nm[host][proto][port]['name']
            state = nm[host][proto][port]['state']
            print(f"Port {port}/{proto}: {service} - {state}")

- Explanation:
  - Nmap SYN Scan: This is a stealth scan that sends SYN packets to detect open ports.
  - Service Detection: Nmap also attempts to detect which services are running on the open ports (e.g., SSH, HTTP).

---

Step 5: Query Vulners API for Vulnerabilities
- Why?: Now that you’ve identified open services with Nmap, you can query the Vulners API to check for vulnerabilities related to these services.

- Instructions: For each detected service, the Vulners API is queried, and the related vulnerability data (CVE IDs, descriptions, severity levels) is retrieved and displayed.

In [ ]:
# Query Vulners for vulnerabilities based on detected services
for host in nm.all_hosts():
    for proto in nm[host].all_protocols():
        ports = nm[host][proto].keys()
        for port in ports:
            service = nm[host][proto][port]['name']
            print(f"\nChecking vulnerabilities for service: {service} on port {port}")
            
            # Use Vulners API to search for vulnerabilities related to the service
            try:
                vuln_data = vulners_api.find_all(f"service:{service}")
                if vuln_data:
                    print(f"Vulnerabilities found for {service} on port {port}:")
                    print(json.dumps(vuln_data, indent=2))  # Print vulnerability data in JSON format
                else:
                    print(f"No vulnerabilities found for {service}.")
            except Exception as e:
                print(f"Error querying vulnerabilities for {service}: {str(e)}")


- Explanation: The code loops through the open services detected by Nmap and queries the Vulners database for related vulnerabilities (e.g., CVEs, exploits, patches).


---

Step 6: Generate Reports (Full Vulnerability Report)
- Why?: Once vulnerabilities are found, it’s important to store the results in a full report that can be used for analysis or remediation planning.

- Instructions: The following code saves all detected vulnerabilities into a JSON file for further analysis.

In [ ]:
import nmap
import vulners
import json
import datetime

# Initialize Nmap and Vulners API
nm = nmap.PortScanner()
vulners_api = vulners.VulnersApi(api_key='YOUR_VULNERS_API_KEY')

# Generate and save the vulnerability report in a JSON file
report_data = {}
report_data['timestamp'] = datetime.datetime.now().isoformat()

for host in nm.all_hosts():
    host_data = {}
    for proto in nm[host].all_protocols():
        host_data[proto] = {}
        ports = nm[host][proto].keys()
        for port in ports:
            service = nm[host][proto][port]['name']
            try:
                if service:
                    vuln_data = vulners_api.find_all(f"service:{service}")
                    if vuln_data:
                        host_data[proto][port] = vuln_data
                    else:
                        host_data[proto][port] = []
            except Exception as e:
                print(f"Error querying vulnerabilities for service {service} on port {port}: {str(e)}")
    
    report_data[host] = host_data

# Save the full report to a JSON file
with open('vulnerability_report.json', 'w') as report_file:
    json.dump(report_data, report_file, indent=4)

print("Vulnerability report saved as 'vulnerability_report.json'.")


- Explanation: This generates a full vulnerability report in JSON format, which contains all the detected vulnerabilities from the scan.

---

Step 7: Service-Specific Vulnerability Reports
- Why?: If you want to focus on specific services (e.g., SSH, HTTP), you can generate service-specific reports to dive deeper into their vulnerabilities.

- Instructions: Modify the service_to_check variable to focus on a specific service.

In [ ]:
# Generate service-specific report (e.g., for SSH)
service_to_check = 'ssh'  # Change this to the service you want to focus on

print(f"\nGenerating report for service: {service_to_check}")

for host in nm.all_hosts():
    for proto in nm[host].all_protocols():
        ports = nm[host][proto].keys()
        for port in ports:
            service = nm[host][proto][port]['name']
            if service == service_to_check:
                print(f"\nChecking vulnerabilities for {service} on port {port}")
                try:
                    # Use Vulners API to search for vulnerabilities related to the service
                    vuln_data = vulners_api.find_all(f"service:{service}")
                    if vuln_data:
                        print(f"Vulnerabilities found for {service} on port {port}:")
                        print(json.dumps(vuln_data, indent=2))  # Print vulnerability data in JSON format
                    else:
                        print(f"No vulnerabilities found for {service}.")
                except Exception as e:
                    print(f"Error querying vulnerabilities for {service} on port {port}: {str(e)}")



##### Summary: Vulnerability Analysis with Nmap and Vulners

This Jupyter notebook demonstrates how to conduct a comprehensive vulnerability analysis by combining Nmap for network scanning and the Vulners API for vulnerability lookup. The notebook walks through each step, from identifying open ports and services using Nmap, to querying Vulners for known vulnerabilities associated with those services. The result is a set of detailed reports highlighting potential security risks, CVEs, exploitability, and remediation suggestions.

Key aspects covered in this analysis include:

- Running Nmap scans to detect open ports and services on target systems.
- Querying the Vulners API to look up vulnerabilities for detected services and software.
- Generating different types of vulnerability reports, such as full vulnerability reports, service-specific reports, and severity-based reports.
- Exporting reports in formats such as JSON and CSV for deeper analysis or integration into security workflows.
This approach allows security analysts and system administrators to automate the detection and prioritization of vulnerabilities in their networks. By using a combination of widely used tools like Nmap and Vulners, the notebook provides a free, customizable, and effective solution for vulnerability assessment and remediation planning.

#### Next Steps after Nmap + Vulners Scan
1. Save the Vulnerability Report:

  - first thing to do is save the vulnerability data from your Nmap and Vulners scan to a file (e.g., JSON). This allows you to easily inspect the data and use it for further analysis.
  
2. Inspect the JSON Report:
  - Once the scan completes, the vulnerability data is stored in vulners_vulnerability_report.json. This file contains detailed information about the detected services, ports, and associated vulnerabilities.
  - You can open this JSON file and inspect the vulnerability data to ensure everything is captured properly.
  
3. Visualize the Vulnerability Data:
  -  Now that the vulnerabilities are saved, you can create visualizations to better understand the results.
  - Visualizing CVSS scores: Plot a histogram of the Common Vulnerability Scoring System (CVSS) scores to see the distribution of vulnerabilities by severity.
  - Visualizing vulnerabilities by severity: Create a bar chart that shows how many vulnerabilities fall into each severity category (e.g., critical, high, medium, low).

Step 1: Save the Vulnerability Report

- Here’s a recap of the code to save the vulnerability report to a JSON file.

In [ ]:
import json

# Initialize report data dictionary
vuln_report = {}

# Generate service-specific report (e.g., for SSH)
service_to_check = 'ssh'  # Change this to the service you want to focus on

print(f"\nGenerating report for service: {service_to_check}")

for host in nm.all_hosts():
    host_data = {}
    for proto in nm[host].all_protocols():
        host_data[proto] = {}
        ports = nm[host][proto].keys()
        for port in ports:
            service = nm[host][proto][port]['name']
            if service == service_to_check:
                print(f"\nChecking vulnerabilities for {service} on port {port}")
                try:
                    # Use Vulners API to search for vulnerabilities related to the service
                    vuln_data = vulners_api.find_all(f"service:{service}")
                    if vuln_data:
                        host_data[proto][port] = vuln_data  # Store vulnerabilities
                        print(f"Vulnerabilities found for {service} on port {port}")
                    else:
                        host_data[proto][port] = "No vulnerabilities found"
                        print(f"No vulnerabilities found for {service}.")
                except Exception as e:
                    print(f"Error querying vulnerabilities for {service} on port {port}: {str(e)}")

    vuln_report[host] = host_data

# Save the vulnerability report as a JSON file
with open('vulners_vulnerability_report.json', 'w') as report_file:
    json.dump(vuln_report, report_file, indent=4)

print("Vulnerability report saved as 'vulners_vulnerability_report.json'.")


---

Step 2: Visualize Vulnerabilities
After generating the JSON report, you can use Python libraries like Matplotlib and Seaborn to visualize the vulnerability data.

Visualizing CVSS Scores:

- This histogram shows how vulnerabilities are distributed by their CVSS score.

In [ ]:
import json

# Load the vulnerability report
with open('vulners_vulnerability_report.json') as report_file:
    vuln_report = json.load(report_file)

# Print a portion of the report to inspect its structure
print(json.dumps(vuln_report, indent=4))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize an empty list to store CVSS scores
cvss_scores = []

# Loop through the vulnerability report to extract CVSS scores
for host, host_data in vuln_report.items():
    for proto, ports_data in host_data.items():
        for port, vulns in ports_data.items():
            if isinstance(vulns, list):  # Ensure it's a list of vulnerabilities
                for vuln in vulns:
                    # Safely extract the CVSS score
                    cvss_score = vuln.get('cvss_score')
                    if cvss_score is not None:
                        cvss_scores.append(cvss_score)
                    else:
                        print(f"No CVSS score found for {vuln.get('cve')}")

# Visualize the CVSS scores using Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(cvss_scores, bins=10, kde=False)
plt.title('Distribution of CVSS Scores for Vulnerabilities')
plt.xlabel('CVSS Score')
plt.ylabel('Number of Vulnerabilities')
plt.show()


Visualizing Vulnerabilities by Severity:
- This bar chart breaks down vulnerabilities by severity level (e.g., critical, high, medium, low).

Dictionary to track the number of vulnerabilities by severity level

In [ ]:
# Dictionary to track the number of vulnerabilities by severity level
severity_levels = {
    "critical": 0,
    "high": 0,
    "medium": 0,
    "low": 0,
}

# Parse the report and count vulnerabilities by severity
for host, host_data in vuln_report.items():
    for proto, ports_data in host_data.items():
        for port, vulns in ports_data.items():
            if isinstance(vulns, list):  # Ensure it's a list of vulnerabilities
                for vuln in vulns:
                    # Safely extract severity and count it
                    severity = vuln.get('severity', '').lower()
                    if severity in severity_levels:
                        severity_levels[severity] += 1

# Plot the severity levels
labels = severity_levels.keys()
counts = severity_levels.values()

plt.figure(figsize=(8, 5))
sns.barplot(x=list(labels), y=list(counts))
plt.title('Vulnerabilities by Severity Level')
plt.xlabel('Severity')
plt.ylabel('Number of Vulnerabilities')
plt.show()


---

Additional Visualization Options

You can further customize the visualizations:
  
- Pie chart of vulnerabilities by severity: Show proportions of critical, high, medium, and low vulnerabilities.
- Time series visualization: If you have historical scan data, plot how vulnerabilities evolve over time.
- Heatmap: Use a heatmap to show which hosts have the highest concentrations of vulnerabilities.

---

Final Steps:
1. Save the JSON report from the scan.
2. Use Matplotlib and Seaborn to create insightful visualizations based on the vulnerability data.
3. Inspect the charts and identify areas for remediation.
   
By following these next steps after running your Nmap + Vulners scan, you can visualize the most critical vulnerabilities and take the appropriate security actions.

Let me know if you need further help with the visualizations or any additional customizations!